In [105]:
import einops
import torch

from einops.layers.torch import EinMix, Rearrange

from utils.components import *
from utils.config import *

In [168]:
ks = (1, 3, 3)
4 > len(ks) > 0

k = len(ks)
assert 4 > k > 0, 'kernel must be 1, 2 or 3d'
conv_fn = getattr(torch.nn, f"Conv{k}d")

In [169]:
network = NetworkConfig(dim=512, num_latents=32, num_layers=12, num_tails=1, dim_out=32, dim_coords=64)
world = WorldConfig(field_sizes={"v": 9, "t": 36, "h": 64, "w": 120}, patch_sizes={'vv': 1, 'tt': 6, 'hh': 8, 'ww': 8}, batch_size=1)


In [ ]:
class ConvNextBlock(torch.nn.Module):
    def __init__(self, dim: int, kernel_size: tuple, num_groups: int = 1, expansion_factor: int = 4):
        super().__init__()
        k = len(kernel_size)
        assert 4 > k > 0, 'kernel must be 1, 2 or 3d'
        conv_fn = getattr(torch.nn, f"Conv{k}d")

        self.block = torch.nn.Sequential()

        self.block.add_module(
            'norm',
            torch.nn.GroupNorm(num_groups= num_groups, num_channels= dim)
        )

        self.block.add_module(
            'dw',
            conv_fn(
                in_channels = dim,
                out_channels = dim,
                padding = 'same',
                kernel_size = kernel_size,
                groups = dim,
            ))

        self.block.add_module(
            'pw_in',
            conv_fn(
                in_channels = dim,
                out_channels = dim * expansion_factor,
                kernel_size = 1,
                groups = num_groups,
            )
        )

        self.block.add_module('silu',torch.nn.SiLU())

        self.block.add_module(
            'pw_out',
            conv_fn(
                in_channels = dim * expansion_factor,
                out_channels = dim,
                kernel_size = 1,
                groups = num_groups,
            )
        )

    def forward(self, x: torch.FloatTensor):
        return self.block(x) + x
    
class ConvInterpolate(torch.nn.Module):
    def __init__(self, dim: int, kernel_size: tuple, out_size: tuple, mode: str = "nearest-exact"):
        super().__init__()
        self._out_size = out_size
        self._mode = mode
        k = len(kernel_size)
        assert 4 > k > 0, 'kernel must be 1, 2 or 3d'
        conv_fn = getattr(torch.nn, f"Conv{k}d")
        self.conv = conv_fn(
                in_channels = dim,
                out_channels = dim,
                padding = 'same',
                kernel_size = kernel_size,
                groups = dim,
            )
            
    def forward(self, x: torch.FloatTensor):
        x = torch.nn.functional.interpolate(x, size = self._out_size, mode = self._mode)
        x = x + self.conv(x)
        return x
    

In [ ]:
class EinDecoder(torch.nn.Module):
    def __init__(self, network: NetworkConfig, world: WorldConfig, depth: int = 0):
        super().__init__()
        # config attributes 
        c = network.dim_out
        k = default(network.num_tails, 1)
        vv = world.patch_sizes['vv']
        v = world.token_sizes['v']
        groups = v * vv * k

        # project tokens to low dimensional space before interpolation
        self.token_to_grid = EinMix(
            pattern=f"b {world.flat_token_pattern} d -> b (v vv k c) t h w",
            weight_shape=f"v vv k c d",
            d = network.dim, c = c, k = k, vv = vv,
            **world.token_sizes
        )

        # small CNN for post-processing
        if depth >= 1:
            self.cnn = torch.nn.Sequential(*[ConvNextBlock(c * groups, (3, 7, 7), groups) for _ in range(depth)])  
        else:
            self.cnn = None

        # unmix grid via interpolate + depthwise separable convolution
        self.upsample = ConvInterpolate(c * groups, (3, 5, 5), out_size= tuple(world.field_sizes[ax] for ax in ['t', 'h', 'w']), mode='nearest-exact')
        
        self.grid_to_field = EinMix(
            f'b (v vv k c) (t tt) (h hh) (w ww) -> b {world.field_pattern} k',
            weight_shape="v vv k c",
            k = k, c = c, **world.patch_sizes, **world.token_sizes
        )

    def forward(self, x: torch.FloatTensor):
        x = self.token_to_grid(x)
        if exists(self.cnn):
            x = self.cnn(x)
        x = self.upsample(x)
        x = self.grid_to_field(x)
        return x

In [217]:
import einops
import torch

from einops.layers.torch import EinMix

from utils.components import *
from utils.config import *
from utils.random_fields import RandomField

class EinMask(torch.nn.Module):
    def __init__(self, network: NetworkConfig, world: WorldConfig):
        super().__init__()
        # store configs
        self.network = network
        self.world = world

        # I/O
        self.to_tokens = EinMix(
            pattern=f"b {world.field_pattern} -> b {world.flat_token_pattern} d", 
            weight_shape=f'{world.patch_pattern} v d', 
            d = network.dim, 
            **world.patch_sizes, **world.token_sizes
            )
        
        self.to_fields = EinDecoder(network, world)
                
        # noise mapping
        if default(network.num_tails, 1) > 1:
            self.noise_generator = None
        else:
            self.noise_generator = RandomField(network.dim, world, has_ffn=False)
        
        # positional embeddings
        self.src_positions = ContinuousPositionalEmbedding(
            dim_per_coord=network.dim_coords, 
            wavelengths=[(1, 2 * k) for k in world.token_shape],
            model_dim=network.dim
        )

        self.tgt_positions = ContinuousPositionalEmbedding(
            dim_per_coord=network.dim_coords, 
            wavelengths=[(1, 2 * k) for k in world.token_shape],
            model_dim=network.dim
        )
        
        # pre-computed coordinates
        self.register_buffer('indices', torch.arange(world.num_tokens))
        self.register_buffer("coordinates", torch.stack(
            torch.unravel_index(indices = self.indices, shape = world.token_shape), 
            dim = -1)
            )        
        
        # learnable latents
        self.latents = torch.nn.Embedding(network.num_latents, network.dim)

        # latent transformer
        self.encoder = torch.nn.ModuleList([
            TransformerBlock(network.dim, dim_ctx=network.dim_noise)
            for _ in range(default(network.num_read_blocks, 1))
        ])

        self.processor = torch.nn.ModuleList([
            TransformerBlock(network.dim, drop_path=network.drop_path, dim_ctx=network.dim_noise)
            for _ in range(default(network.num_compute_blocks, 1))
        ])
        
        self.decoder = torch.nn.ModuleList([
            TransformerBlock(network.dim, dim_ctx=network.dim_noise, has_skip= n > 0)
            for n in range(default(network.num_write_blocks, 1))
        ])

        # Weight initialization
        self.apply(self.base_init)

    @staticmethod
    def base_init(m: torch.nn.Module):
        # linear
        if isinstance(m, torch.nn.Linear):
            torch.nn.init.trunc_normal_(m.weight, std = 0.02)
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias)
        # embedding
        if isinstance(m, torch.nn.Embedding):
            torch.nn.init.trunc_normal_(m.weight, std = 0.02)
        # einmix
        if isinstance(m, EinMix):
            torch.nn.init.trunc_normal_(m.weight, std = 0.02)
            if m.bias is not None:
                torch.nn.init.trunc_normal_(m.bias, std = 0.02)
        # convolution
        if isinstance(m, torch.nn.Conv3d):
            torch.nn.init.zeros_(m.weight)
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias)
        # conditional layer norm
        if isinstance(m, ConditionalLayerNorm):
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias)
            if m.weight is not None: # CLN weight close to 0
                torch.nn.init.trunc_normal_(m.weight, std = 1e-7)
    
    def forward(self, 
                fields: torch.FloatTensor, 
                srcs: List[torch.LongTensor] | torch.LongTensor, 
                tgts: List[torch.LongTensor] | torch.LongTensor = None,
                members: Optional[int] = None, 
                rng: Optional[torch.Generator] = None
                ) -> torch.FloatTensor:
        B = fields.size(0)
        D = self.network.dim
        K = default(self.network.num_tails, 1)
        E = default(members, 1)

        # expand to ensemble form
        fields = einops.repeat(fields, "b ... -> (b e) ...", e = E, b = B)
        latents = einops.repeat(self.latents.weight, '... -> (b e) ...', b = B, e = E)
        coo = einops.repeat(self.coordinates, '... -> (b e) ...', b = B, e = E)
        src_idx = einops.repeat(srcs, 'b ... -> (b e) ... d', d = D, e = E, b = B)
        
        # embed full fields as tokens
        tokens = self.to_tokens(fields).gather(1, src_idx)

        # prepare tgt and src
        tgt = self.tgt_positions(coo)
        src = self.src_positions(coo).scatter_add_(1, src_idx, tokens)

        # maybe add random field
        if exists(self.noise_generator):
            noise = self.noise_generator(shape = (B * E,), rng = rng).to(src.dtype)
            tgt = tgt + noise
            src = src + noise

        # map ctx to latents
        for read in self.encoder:
            latents = read(q = latents, kv = torch.cat([src, latents], dim = 1))

        # process latents
        for process in self.processor:
            latents = process(q = latents)

        # map latents to tgt
        for write in self.decoder:
            tgt = write(q = tgt, kv = latents)

        # map all tokens back to fields
        fields = self.to_fields(tgt)
        
        # rearrange to ensemble form
        fields = einops.rearrange(fields, "(b e) ... k -> b ... (e k)", e = E, b = B, k = K)
        return fields

In [218]:
model = EinMask(network=network, world=world)

In [219]:
x = torch.randn(world.batch_size, *world.field_shape)
m = torch.multinomial(torch.ones(world.batch_size, world.num_tokens), 512)

y = model(x, m)